# Predict Winner from Realtime Stats using RNN

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.autograd import Variable
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 0
MAX_TIME_STEP = 10

## Dataset

In [3]:
df = pd.read_csv('../data/LeagueofLegends.csv', sep=',')
df = df[df['gamelength'] >= MAX_TIME_STEP]
df.reset_index(drop = True, inplace = True)
matches = len(df)
print(f'# of matches: {matches}')

from ast import literal_eval
df['golddiff'] = df['golddiff'].apply(literal_eval)
df[['golddiff']].head()

# of matches: 7620


,golddiff
0,"[0, 0, -14, -65, -268, -431, -488, -789, -494,..."
1,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,..."
2,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ..."
3,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ..."
4,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20..."


In [4]:
def count_item(items):
    count = np.zeros(MAX_TIME_STEP, dtype=np.int8)
    for timestep in range(MAX_TIME_STEP) :
        for item in items:
            if item[0] <= timestep + 1:
                count[timestep] += 1
    return count

df['bDragons'] = df['bDragons'].apply(literal_eval)
df['rDragons'] = df['rDragons'].apply(literal_eval)

df['bDragons'] = df['bDragons'].apply(count_item)
df['rDragons'] = df['rDragons'].apply(count_item)
df['dragondiff'] = df['bDragons'] - df['rDragons']

df[['dragondiff']].head()

,dragondiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [5]:
df['bBarons'] = df['bBarons'].apply(literal_eval)
df['rBarons'] = df['rBarons'].apply(literal_eval)

df['bBarons'] = df['bBarons'].apply(count_item)
df['rBarons'] = df['rBarons'].apply(count_item)
df['barondiff'] = df['bBarons'] - df['rBarons']

df[['barondiff']].head()

,barondiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [6]:
df['bHeralds'] = df['bHeralds'].apply(literal_eval)
df['rHeralds'] = df['rHeralds'].apply(literal_eval)

df['bHeralds'] = df['bHeralds'].apply(count_item)
df['rHeralds'] = df['rHeralds'].apply(count_item)
df['heralddiff'] = df['bHeralds'] - df['rHeralds']

df[['heralddiff']].head()

,heralddiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [7]:
df['bTowers'] = df['bTowers'].apply(literal_eval)
df['rTowers'] = df['rTowers'].apply(literal_eval)

df['bTowers'] = df['bTowers'].apply(count_item)
df['rTowers'] = df['rTowers'].apply(count_item)
df['towerdiff'] = df['bTowers'] - df['rTowers']

df[['towerdiff']].head()

,towerdiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [8]:
df['bInhibs'] = df['bInhibs'].apply(literal_eval)
df['rInhibs'] = df['rInhibs'].apply(literal_eval)

df['bInhibs'] = df['bInhibs'].apply(count_item)
df['rInhibs'] = df['rInhibs'].apply(count_item)
df['inhibitordiff'] = df['bInhibs'] - df['rInhibs']

df[['inhibitordiff']].head()

,inhibitordiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [9]:
df['bKills'] = df['bKills'].apply(literal_eval)
df['rKills'] = df['rKills'].apply(literal_eval)

df['bKills'] = df['bKills'].apply(count_item)
df['rKills'] = df['rKills'].apply(count_item)
df['killdiff'] = df['bKills'] - df['rKills']

df[['killdiff']].head()

,killdiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0, 1, 2, 2, 1, 1]"
3,"[0, 0, 0, 0, 0, 0, 0, -1, 0, -1]"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [10]:
stats = ['golddiff','dragondiff', 'barondiff', 'heralddiff', 'towerdiff', 'inhibitordiff', 'killdiff']
x = df[stats]
y = df['bResult']

x.tail()

,golddiff,dragondiff,barondiff,heralddiff,towerdiff,inhibitordiff,killdiff
7615,"[0, 0, -18, -95, 45, -87, -117, 199, 126, 92, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 2]"
7616,"[0, 0, -86, -39, -207, -349, -60, -140, 187, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, -1, -1, -1, -1, -1, -2, -2]"
7617,"[0, -8, -6, 116, 103, -92, -470, -958, -1998, ...","[0, 0, 0, 0, 0, 0, 0, 0, -1, -1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, -1, -2, -2, -2]"
7618,"[0, 0, -97, 33, 351, 284, 299, 263, 403, 623, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1]"
7619,"[0, 0, -8, -225, -36, 73, 464, 184, 1171, 1409...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]"


Normalize data:

In [11]:
from sklearn.preprocessing import StandardScaler

data = {}
scalers = {}
for stat in stats:
    scalers[stat] = StandardScaler()
    for row in df[stat]:
        scalers[stat].partial_fit(np.asanyarray(row).reshape(-1, 1))
    data[stat] = [scalers[stat].transform(np.asanyarray(row).reshape(-1, 1)).reshape(-1) for row in df[stat]]

num_features = len(data)
print(f'# of features per timestep: {num_features}')

# of features per timestep: 7


Build dataset:

In [12]:
class LOLDataset(Dataset):
    def __init__(self, data, stats, label):
        
        self.data =[]
        for t in range(MAX_TIME_STEP):
            self.data.append([[data[stat][i][t] for stat in stats] for i in range(matches)])
        # print("shape:", np.asanyarray(self.data).shape)
        self.label=[i for i in label]
        
    
    def __getitem__(self, item):
        return torch.tensor([ [torch.scalar_tensor(i) for i in x[item]] for x in self.data]), torch.tensor(self.label[item])

    def __len__(self):
        return len(self.label)
            

## Neural Network Structure

### Recurrent Neural Network (RNN)

In [13]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()
        self.hidden_size = 256
        
        self.rnn= nn.RNN(
            nonlinearity = 'relu',
            input_size = num_features,
            hidden_size = self.hidden_size,
            num_layers = 1,
            batch_first = True
        )

        self.out = nn.Linear(self.hidden_size, 2)
    
    def forward(self,x):
        r_out, hn = self.rnn(x, torch.zeros(1, len(x), self.hidden_size))
        out = self.out(r_out[:, -1, :])
        return out


#### Long Short-Term Memory (LSTM) Network

In [14]:
class LSTMRNN(nn.Module):
    def __init__(self):
        super(LSTMRNN, self).__init__()
        self.hidden_size = 256
        self.lstm = nn.LSTM(
            input_size = num_features,
            hidden_size = self.hidden_size,
            num_layers = 1,
            batch_first = True
        )
        self.out = nn.Linear(self.hidden_size, 2)
    
    def forward(self,x):
        h0 = torch.zeros(1, len(x), self.hidden_size)
        c0 = torch.zeros(1, len(x), self.hidden_size)
        r_out, (h_n, h_c) = self.lstm(x, (h0, c0))
        out = self.out(r_out[:, -1, :])
        return out

## Training

Split the dataset into Train:Held-Out:Test = 6:2:2, so we can early stop when held-out accuracy drops.

Using a batch size of 32 to load dataset for training.

In [15]:
BATCH_SIZE = 32

dataset = LOLDataset(data, stats, df["bResult"])
test_size = valid_size = int(0.2 * len(dataset))
train_size = len(dataset) - test_size - valid_size

trainDataset, validDataset, testDataset = random_split(
    dataset = dataset,
    lengths = [train_size, valid_size, test_size],
    generator = torch.Generator().manual_seed(0)
)

trainLoader = DataLoader(trainDataset, batch_size = BATCH_SIZE, shuffle=True)
validLoader = DataLoader(validDataset, batch_size = BATCH_SIZE)
testLoader = DataLoader(testDataset, batch_size = BATCH_SIZE)

In [16]:
def train(dataloader, model, loss_fn, optimizer, mute = False):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = Variable(x), Variable(y)

        predict = model(x)
        loss = loss_fn(predict, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 30 == 0 and not mute:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [17]:
def test(dataloader, model, loss_fn, validation = False):
    model.eval()
    size = len(dataloader.dataset)

    correct = 0
    test_loss = 0
    with torch.no_grad():
        for step,(x,y) in enumerate(dataloader):
            x, y = Variable(x), Variable(y)
            predict = model(x)
            # print(predict)
            test_loss += loss_fn(predict, y).item()
            correct += (predict.argmax(1) == y).sum().item()
    
    print(f"{'Valid' if validation else 'Test'} Acc:{correct/size:>7f}, Avg Loss: {test_loss/size:>7f}")
    return correct/size

### Training RNN

In [18]:
MUTE = False
EPOCH = 100
LR = 0.001

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

model_RNN = RNN()
print(model_RNN)

optimizer = torch.optim.Adam(model_RNN.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

best_acc = 0
early_stopping = 0
early_stopping_threshold = 10

for epoch in range(1, EPOCH + 1):
    print(f"--------- Epoch #{epoch} ---------")
    train(trainLoader, model_RNN, loss_func, optimizer, mute = MUTE)
    valid_acc = test(validLoader, model_RNN, loss_func, validation = True)
    if valid_acc > best_acc :
        early_stopping = 0
        best_acc = valid_acc
        torch.save(model_RNN.state_dict(), "./model/RNN_"+str(MAX_TIME_STEP)+".pt")
    else :
        early_stopping += 1
        if early_stopping == early_stopping_threshold :
            print(f"Early stopped at epoch #{epoch} with best validation accuracy {best_acc*100:.2f}%.")
            break


RNN(
  (rnn): RNN(7, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)
--------- Epoch #1 ---------
loss: 0.672370  [    0/ 4572]
loss: 0.603202  [  960/ 4572]
loss: 0.627706  [ 1920/ 4572]
loss: 0.574753  [ 2880/ 4572]
loss: 0.592131  [ 3840/ 4572]
Valid Acc:0.673885, Avg Loss: 0.018991
--------- Epoch #2 ---------
loss: 0.627097  [    0/ 4572]
loss: 0.667539  [  960/ 4572]
loss: 0.633175  [ 1920/ 4572]
loss: 0.612359  [ 2880/ 4572]
loss: 0.655088  [ 3840/ 4572]
Valid Acc:0.682415, Avg Loss: 0.018849
--------- Epoch #3 ---------
loss: 0.541198  [    0/ 4572]
loss: 0.597546  [  960/ 4572]
loss: 0.699851  [ 1920/ 4572]
loss: 0.472916  [ 2880/ 4572]
loss: 0.599682  [ 3840/ 4572]
Valid Acc:0.681102, Avg Loss: 0.018576
--------- Epoch #4 ---------
loss: 0.527057  [    0/ 4572]
loss: 0.595158  [  960/ 4572]
loss: 0.577839  [ 1920/ 4572]
loss: 0.533903  [ 2880/ 4572]
loss: 0.586298  [ 3840/ 4572]
Valid Acc:0.683727, Avg Loss: 0.018377
--------- Epoch #5 ---

### Training LSTM RNN

In [25]:
model_LSTM = LSTMRNN()
print(model_LSTM)

LR = 0.0001
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

optimizer = torch.optim.Adam(model_LSTM.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

best_acc = 0
early_stopping = 0
early_stopping_threshold = 5

for epoch in range(1, EPOCH + 1):
    print(f"--------- Epoch #{epoch} ---------")
    train(trainLoader, model_LSTM, loss_func, optimizer, mute = MUTE)
    valid_acc = test(validLoader, model_LSTM, loss_func, validation = True)
    if valid_acc > best_acc :
        early_stopping = 0
        best_acc = valid_acc
        torch.save(model_LSTM.state_dict(), "./model/LSTM_"+str(MAX_TIME_STEP)+".pt")
    else :
        early_stopping += 1
        if early_stopping == early_stopping_threshold :
            print(f"Early stopped at epoch #{epoch} with best validation accuracy {best_acc * 100:.2f}%.")
            break

LSTMRNN(
  (lstm): LSTM(7, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)
--------- Epoch #1 ---------
loss: 0.706780  [    0/ 4572]
loss: 0.684647  [  960/ 4572]
loss: 0.675045  [ 1920/ 4572]
loss: 0.625593  [ 2880/ 4572]
loss: 0.657522  [ 3840/ 4572]
Valid Acc:0.654199, Avg Loss: 0.019831
--------- Epoch #2 ---------
loss: 0.676804  [    0/ 4572]
loss: 0.639502  [  960/ 4572]
loss: 0.762702  [ 1920/ 4572]
loss: 0.792997  [ 2880/ 4572]
loss: 0.608176  [ 3840/ 4572]
Valid Acc:0.657480, Avg Loss: 0.019492
--------- Epoch #3 ---------
loss: 0.632968  [    0/ 4572]
loss: 0.715139  [  960/ 4572]
loss: 0.638977  [ 1920/ 4572]
loss: 0.583995  [ 2880/ 4572]
loss: 0.671027  [ 3840/ 4572]
Valid Acc:0.660761, Avg Loss: 0.019376
--------- Epoch #4 ---------
loss: 0.566009  [    0/ 4572]
loss: 0.580121  [  960/ 4572]
loss: 0.583301  [ 1920/ 4572]
loss: 0.635927  [ 2880/ 4572]
loss: 0.502541  [ 3840/ 4572]
Valid Acc:0.662073, Avg Loss: 0.019300
--------- Epoch 

## Test

Load best models for both RNN and LSTM and test for results:

In [26]:
print("RNN:")
model_RNN.load_state_dict(torch.load("./model/RNN_"+str(MAX_TIME_STEP)+".pt"))
acc_RNN = test(testLoader, model_RNN, loss_func)

print("LSTM RNN:")
model_LSTM.load_state_dict(torch.load("./model/LSTM_"+str(MAX_TIME_STEP)+".pt"))
acc_LSTM = test(testLoader, model_LSTM, loss_func)

print(f'RNN Accuracy = {acc_RNN*100:>.2f}% | LSTM Accuracy = {acc_LSTM*100:>.2f}%')

RNN:
Test Acc:0.665354, Avg Loss: 0.018905
LSTM RNN:
Test Acc:0.666667, Avg Loss: 0.019085
RNN Accuracy = 66.54% | LSTM Accuracy = 66.67%


Predict S11 World Championship Final: EDG vs DK

In [27]:
match_stats = {}

''' S11 EDG vs DK, match 3, DK(red) wins '''
match_stats['golddiff'] = [0,31,16,122,-71,325,170,367,463,918,1479,1181,689,813,890,1486,2779,2215,2543,2805,2887,2975,3477,3842,3361,3450,3318,3101,754,752]
match_stats['dragondiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -2, -2, -2, -2, -2, -2, -3, -3, -3, -3, -3, -3, -4, -4]
match_stats['barondiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
match_stats['heralddiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
match_stats['towerdiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2]
match_stats['inhibitordiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
match_stats['killdiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -1, -1]

''' S11 EDG vs DK, match 4, EDG(blue) wins '''
# match_stats['golddiff'] = [0,-33,147,160,-124,-627,621,979,577,759,1136,1175,-39,554,2057,1839,2071,1771,2058,2515,2297,1408,1575,1459,1521,1525,2672,2314,2717,2417,]
# match_stats['dragondiff'] = [0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,4,4,4,4]
# match_stats['barondiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# match_stats['heralddiff'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
# match_stats['towerdiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1]
# match_stats['inhibitordiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# match_stats['killdiff'] = [0,0,0,0,0,0,1,1,1,1,1,1,-1,-1,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1]


''' S11 DK vs EDG, match 5, EDG(red) wins ''' 
# match_stats['golddiff'] = [0,-21,-108,135,125,451,233,133,-431,-810,-860,-1128,-794,-774,-797,-886,-401,-216,-321,35,-143,-55,-734,-1719,-1955,-1864,-2100,-487,-324,-3383]
# match_stats['dragondiff'] = [0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,-1,-1,-1,-1,-1,-2,-2,-2,-2,-2,-2,-3]
# match_stats['barondiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1]
# match_stats['heralddiff'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
# match_stats['towerdiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,-1]
# match_stats['inhibitordiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# match_stats['killdiff'] = [0,0,0,0,0,0,0,0,-1,-1,-2,-2,-3,-3,-3,-2,-2,-2,-2,-2,-2,-2,-5,-6,-6,-6,-6,-6,-6,-10]

for stat in stats:
    match_stats[stat] = scalers[stat].transform(np.asanyarray(match_stats[stat]).reshape(-1, 1)).reshape(-1)

x = np.asarray([[ [match_stats[stat][timestep] for stat in stats] for timestep in range(MAX_TIME_STEP) ]], dtype=np.float32)

print(x.shape)

def predict_one_match(model, x):
    model.eval()
    with torch.no_grad():
        x = torch.from_numpy(x)
        predict = model(x)
        # print(predict)
        winner = ['red', 'blue'][predict.argmax(1)]
        from math import exp
        prob_red = exp(predict[0][0].item()) / (exp(predict[0][0].item()) + exp(predict[0][1].item()))
        prob_blue = exp(predict[0][1].item()) / (exp(predict[0][0].item()) + exp(predict[0][1].item()))
        print(f"model predicted winner: { winner }")
        print(f"red wins: {prob_red * 100 :.1f}% | blue wins: {prob_blue * 100:.1f}%")

print("RNN:")
predict_one_match(model_RNN, x)
print("LSTM RNN:")
predict_one_match(model_LSTM, x)

(1, 10, 7)
RNN:
model predicted winner: blue
red wins: 25.4% | blue wins: 74.6%
LSTM RNN:
model predicted winner: blue
red wins: 38.0% | blue wins: 62.0%
